# Single campaign design

In [ ]:
import numpy as np
import pandas as pd

from pipes.design.single_campaign_pipe import SingleCmpDesign
from utils.tools import split_cgpg, prep_stratification_column
from utils.db_conn import connect_to_db

import matplotlib.pyplot as plt
%matplotlib inline

## Connect to DB

In [ ]:
db_config = connect_to_db()

## Uplift model

### Set params

In [ ]:
config_path = 'config_s_2.yml'

### Load config

In [ ]:
scd = SingleCmpDesign(config_path)

In [ ]:
scd.cmp_config

In [ ]:
le_channel = scd.label_encoders['channel']

In [ ]:
sprint_number = scd.cmp_config['campaign_name'][1:3]
campaign_name = scd.cmp_config['campaign_name']
audience_id = scd.cmp_config['campaign_name'] + '_uplift'
frequency_level = scd.cmp_config['frequency_level']
start_date = str(scd.cmp_config['start_date'])
product_id = scd.cmp_config['product_id']

In [ ]:
uplift_size = 20_000
pg_size = 10_000
cg_size = 10_000

### Get members for campaign

In [ ]:
sql = f'''
exec Marketing.cvm.sp_membersForCampaigns
    @notificationType = 'buoy',            -- announcement | buoy
    @audience_id = '{audience_id}',        -- номер кампании указанный в беклоге
    @audienceFreq = {frequency_level},     -- необходимая частота для выборки
    @updateE = 0,                          -- исключение сотрудников: 1 - если FTG или Кулинария, 0 - в остальных случаях
    @date = '{start_date}',                -- дата запуска кампании
    @product = '{product_id}';             -- указываем productID, если productID соответсвует запускаемой кампании, если нет - то просто комментируем эту строку

if object_id('Marketing.cvm.campaigns_{campaign_name}_scores_uplift') is not null
   drop table Marketing.cvm.campaigns_{campaign_name}_scores_uplift;
   
select memberId, channel
  into Marketing.cvm.campaigns_{campaign_name}_scores_uplift
  from Marketing.cvm.members_for_{campaign_name}_uplift_campaign;
'''

In [ ]:
db_config['cursor'].execute(sql)

### Load features

In [ ]:
scd.load_data()

### Preprocessing

In [ ]:
scd.preprocess_data()

### Get uplift scores

In [ ]:
scd.get_scores()

### Explore & cut

In [ ]:
final = scd.final_data

In [ ]:
print('Final shape: ', final.shape[0], '\nNumber of unique members: ', final['memberid'].nunique())

In [ ]:
# score_maxes = final.groupby(['memberid'])['uplift_score'].transform(max)
# final = final[final['uplift_score'] == score_maxes].drop_duplicates(subset=['memberid'])

# print('Final shape: ', final.shape[0], '\nNumber of unique members: ', final['memberid'].nunique())

In [ ]:
percent = 0.8
score_interval = np.linspace(0.5, -0.5, 1001)

for threshold in score_interval:
    if final[(final['uplift_score'] > threshold) & (final['model_cg_score'] < percent)].shape[0] > uplift_size * 1.01:
        print('Threshold = ', threshold)
        print(final[(final['uplift_score'] > threshold) & (final['model_cg_score'] < percent)].shape)
        break

In [ ]:
final_cutted = final[(final.avg_cheque >= 1) & (final.avg_cheque <= 2000)].reset_index().drop('index', axis=1)

# Stratify uplift

In [ ]:
final_cutted_2 = final_cutted[(final_cutted.uplift_score > threshold) & (final_cutted['model_cg_score'] < percent)]
final_cutted_2 = final_cutted_2.reset_index().drop('index', axis=1)
indx_uplift = np.random.choice(range(len(final_cutted_2)), uplift_size, replace = False)
final_uplift = final_cutted_2[final_cutted_2.index.isin(indx_uplift)]

final_uplift.shape

In [ ]:
plt.plot(range(0, len(final_uplift)), sorted(final_uplift['uplift_score']))

In [ ]:
final_uplift_cutted = final_uplift[(final_uplift['uplift_score'] > threshold) & (final_uplift['model_cg_score'] < percent)]
final_uplift_cutted = final_uplift_cutted.iloc[:uplift_size]

final_uplift_cutted.shape

In [ ]:
cg, pg = split_cgpg(final_uplift_cutted, pg_size,
                    cont_vars2stratify = ['avg_cheque', 'uplift_score'],
                    descr_vars2stratify = ['channel'])

final_uplift_cutted.loc[cg, 'Group'] = 'CG'
final_uplift_cutted.loc[pg, 'Group'] = 'PG'

pd.pivot_table(final_uplift_cutted, values='uplift_score', index='Group')

In [ ]:
final_uplift_cutted['channel'] = le_channel.inverse_transform(final_uplift_cutted.channel)
final_cutted_uploading = final_uplift_cutted[['memberid', 'channel']].copy()
final_cutted_uploading['audience_id'] = campaign_name + '_uplift'
final_cutted_uploading['is_pg'] = (final_uplift_cutted.Group == 'PG')*1
final_cutted_uploading['group_id'] = final_cutted_uploading.is_pg + 1

final_cutted_uploading.shape

# Save to DB

In [ ]:
final_cutted_uploading.head()

In [ ]:
final_cutted_uploading.shape

In [ ]:
date_today = pd.Timestamp('today')

final_cutted_uploading['date_add'] = '{:%Y-%m-%d}'.format(date_today)
final_cutted_uploading = final_cutted_uploading.rename(columns={'memberid':'memberId'})
final_cutted_uploading[['audience_id',
                        'group_id',
                        'is_pg',
                        'channel',
                        'memberId',
                        'date_add']].to_sql(f'campaigns_to_run_{sprint_number}',
                                            db_config['engine'], schema='Marketing.cvm', index=False, if_exists='append')

# Attach phone numbers and export to Excel

In [ ]:
import os
from utils.pg_design import PreparePilotGroups

In [ ]:
p = PreparePilotGroups(audience_id=audience_id,
                       table_name=f'campaigns_to_run_{sprint_number}',
                       connect_phone=True)

In [ ]:
p.handle_missing_phone(to_do='delete') # delete or 'assign_coupon'

In [ ]:
p.check()

In [ ]:
p.df.groupby(['audience_id', 'group_id', 'channel'])[['mobile_phone']].count()

In [ ]:
group_id_mapping = {
    f'{audience_id}': {
        1: '',
        2: ''
    }
}

In [ ]:
promo_id_mapping={
    f'{audience_id}': {
        1: '',
        2: ''
    }
}

In [ ]:
path = r'D:\o.churiumov\Campaigns\Frequency'+ f'\{audience_id}'
os.mkdir(path)
p.save(folder=path, groups_mapping=group_id_mapping[audience_id], promo_mapping=promo_id_mapping[audience_id])